In [1]:
using DIVAnd
using PyPlot
using PyCall
using Dates
using Statistics
using DelimitedFiles
push!(LOAD_PATH, pwd())
using BlueCloudPlankton
using Test

┌ Info: Precompiling BlueCloudPlankton [top-level]
└ @ Base loading.jl:1260


In [2]:
# Import stuff from cartopy
ccrs = pyimport("cartopy.crs")
gridliner = pyimport("cartopy.mpl.gridliner")
cfeature = pyimport("cartopy.feature")
mticker = pyimport("matplotlib.ticker")
myproj = ccrs.PlateCarree()
coast = cfeature.GSHHSFeature(scale="intermediate");
coast_h = cfeature.GSHHSFeature(scale="high");

mpl = pyimport("matplotlib");
cartopyticker = pyimport("cartopy.mpl.ticker")
lon_formatter = cartopyticker.LongitudeFormatter()
lat_formatter = cartopyticker.LatitudeFormatter()
mpl.rc("axes", linewidth=2)
mpl.rc("font", weight="light", size=14)

In [3]:
datadir = "../data/"
figdir = "../figures"
if !isdir(figdir)
    mkdir(figdir)
end
datafile = joinpath(datadir, "data.csv")
@info(isfile(datafile));

longrid = -90.:0.5:40.
latgrid = 30.:0.5:80.

┌ Info: true
└ @ Main In[3]:7


30.0:0.5:80.0

In [ ]:
function decorate_global_map(ax, coast)
    PyPlot.grid(linewidth=0.2)
    ax.add_feature(coast, color=".6",
            edgecolor="k", zorder=5)
    ax.set_xticks(-180.:45.5:180.)
    ax.set_yticks(-90.:30:90.)
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

end

In [7]:
function decorate_domain_map(ax, coast)
    PyPlot.grid(linewidth=0.2)
    ax.add_feature(coast, color=".6",
            edgecolor="k", zorder=5)
    ax.set_xticks(-90.:20.:40.)
    ax.set_yticks(30.:10.:80.)
    ax.set_xlim(-90., 40.)
    ax.set_ylim(30., 80.)
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

end

decorate_domain_map (generic function with 1 method)

In [ ]:
fig = PyPlot.figure()
ax = subplot(111, projection=myproj)
ax.plot(0., 0., "ko")
decorate_domain_map(ax, coast)

## Read data

In [4]:
lon, lat, dates, abundance, scientificNames = BlueCloudPlankton.read_data(datafile);

In [5]:
namelist = unique(scientificNames)

6-element Array{Any,1}:
 "Metridia lucens"
 "Calanus finmarchicus"
 "Oithona"
 "Temora longicornis"
 "Acartia"
 "Calanus helgolandicus"

## Plots

In [ ]:
fig = PyPlot.figure(figsize=(12, 15))
ax = PyPlot.subplot(111, projection=myproj)
ax.plot(lon, lat, "go", markersize=0.2)
BlueCloudPlankton.decorate_domain_map(ax, coast_h)
PyPlot.savefig(joinpath(figdir, "data_distrib03"), dpi=300, bbox_inches="tight")
PyPlot.close()

### By species

In [7]:
fig = PyPlot.figure(figsize=(15, 12))
for (ii, names) in enumerate(namelist)
    gooddata = findall(scientificNames .== names)
    @info("Found $(length(gooddata)) data points for $(names)")
    ax = PyPlot.subplot(3, 2, ii, projection=myproj)
    ax.plot(0., 0.)
    ax.plot(lon[gooddata], lat[gooddata], "k.", markersize=0.2)
    title("$(names): $(length(gooddata)) data points")
    decorate_domain_map(ax)
end
PyPlot.savefig(joinpath(figdir, "data_species"), dpi=300, bbox_inches="tight")
PyPlot.close()

UndefVarError: UndefVarError: namelist not defined

## Bathymetry

In [8]:
bathname = joinpath(datadir, "gebco_30sec_16.nc")

"../data/gebco_30sec_16.nc"

In [9]:
if !isfile(bathname)
    @info("Download bathymetry")
    download("https://dox.ulg.ac.be/index.php/s/U0pqyXhcQrXjEUX/download",bathname)
else
    @info("Bathymetry file already downloaded")
end

┌ Info: Bathymetry file already downloaded
└ @ Main In[9]:5


In [10]:
bx, by, b = load_bath(bathname, true, longrid, latgrid);

In [11]:
fig = PyPlot.figure(figsize=(10, 10))
ax = PyPlot.subplot(111, projection=myproj)
pcm = ax.pcolor(bx, by, b'); 
colorbar(pcm, orientation="vertical", shrink=0.35)
PyPlot.title("Bathymetry")
PyPlot.savefig(joinpath(figdir, "domain_bathymetry2"), dpi=300, bbox_inches="tight")
PyPlot.close()

### Metrics

In [12]:
_, (pm,pn), (xi,yi) = DIVAnd.DIVAnd_rectdom(longrid, latgrid);

### Land-sea mask

In [13]:
xmask, ymask, mmask = load_mask(bathname, true, longrid, latgrid, 0.0);

In [14]:
fig = PyPlot.figure(figsize=(10, 10))
ax = PyPlot.subplot(111, projection=myproj)
pcm = ax.pcolor(bx, by, transpose(mmask), cmap=PyPlot.cm.binary_r); 
decorate_domain_map(ax)
PyPlot.title("Land-sea mask")
PyPlot.savefig(joinpath(figdir, "domain_landsea_mask2"), dpi=300, bbox_inches="tight")
PyPlot.close()

UndefVarError: UndefVarError: decorate_domain_map not defined

## DIVAnd interpolation
### Set parameters

In [15]:
len = 2.5;
epsilon2 = 10.

10.0

In [16]:
gooddata = findall(scientificNames .== namelist[1])
@info("Found $(length(gooddata)) data points for $(namelist[1])")

UndefVarError: UndefVarError: namelist not defined

In [17]:
llon, llat = ndgrid(longrid, latgrid);

In [18]:
for (ii, names) in enumerate(namelist[1])
    gooddata = findall(scientificNames .== names)
    fi, s = DIVAndrun(mmask, (pm, pn), (llon, llat), 
        (lon[gooddata], lat[gooddata]), abundance[gooddata] .- mean(abundance[gooddata]), 
        (len, len), epsilon2);
    
    fname = "$(replace(namelist[ii], " "=>"_"))_interp"
    ncfilename = "$(replace(namelist[ii], " "=>"_"))_interp.nc"
    #plot_results(longrid, latgrid, fi .+ mean(abundance[gooddata]), 
    #    namelist[ii], joinpath(figdir, fname))
    create_nc_results(ncfilename, longrid, latgrid, field, names)
end 

UndefVarError: UndefVarError: namelist not defined

In [19]:
fname = "$(replace(namelist[1], " "=>"_"))_interp"
plot_results(longrid, latgrid, fi .+ mean(abundance[gooddata]), 
    namelist[1], joinpath(figdir, fname))

UndefVarError: UndefVarError: namelist not defined

In [20]:
function create_nc_results(filename::String, lons, lats, field,
    speciesname::String=""; valex=-999.9)
    Dataset(filename, "c") do ds

        # Dimensions
        ds.dim["lon"] = length(lons)
        ds.dim["lat"] = length(lats)
        ds.dim["time"] = Inf # unlimited dimension

        # Declare variables
        ncfield = defVar(ds,"LOGSPM", Float64, ("lon", "lat", "time"))
        ncfield.attrib["missing_value"] = Float64(valex)
        ncfield.attrib["_FillValue"] = Float64(valex)
        ncfield.attrib["long_name"] = "interpolated abundance"
        ncfield.attrib["units"] = "ind/m3"

        nctime = defVar(ds,"time", Float32, ("time",))
        nctime.attrib["missing_value"] = Float32(valex)
        nctime.attrib["units"] = "seconds since 1981-01-01 00:00:00"
        nctime.attrib["time"] = "time"

        nclon = defVar(ds,"lon", Float32, ("lon",))
        nclon.attrib["missing_value"] = Float32(valex)
        nclon.attrib["_FillValue"] = Float32(valex)
        nclon.attrib["units"] = "degrees East"
        nclon.attrib["lon"] = "longitude"

        nclat = defVar(ds,"lat", Float32, ("lat",))
        nclat.attrib["missing_value"] = Float32(valex)
        nclat.attrib["_FillValue"] = Float32(valex)
        nclat.attrib["units"] = "degrees North"
        nclat.attrib["lat"] = "latitude"

        # Global attributes
        ds.attrib["institution"] = "GHER - University of Liege"
        ds.attrib["title"] = "Interpolated field of $(speciesname)"
        ds.attrib["comment"] = "Original data prepared by VLIZ"
        ds.attrib["author"] = "C. Troupin (ctroupin@uliege), A. Barth (a.barth@uliege.be)"
        ds.attrib["tool"] = "create_nc_spm_tile.jl"
        ds.attrib["institution_url"] = "http://labos.ulg.ac.be/gher/"
        ds.attrib["institution_logo_url"] = "http://gher-diva.phys.ulg.ac.be/Images/gher-logo.png"

        # Define variables
        ncfield[:] = field
        # nctime[:] = times
        nclon[:] = lons
        nclat[:] = lats;

    end
end;